<div style="text-align: right"> Mark Hendricks </div>

<left>FINM 36700 - Portfolio Theory and Risk Management</left> 
<br>
<left>Fall 2022</left>

<h2><center> Midterm #1 </center></h2>

<center>Monday, Oct 17</center>

<h3><span style="color:#00008B">TA Solution</span></h3>


### Please note the following:
- #### The exam is 100 points.
- #### You have 120 minutes to complete the exam.
- #### For every minute late you submit the exam, you will lose one point.
- #### You will upload your solution to the Midterm #1 assignment on Canvas, where you downloaded this.
- #### Your submission should be readable, (the graders can understand your answers,) and it should include all code used in your analysis.
- #### The exam is open-material, closed-communication.
- #### If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
- #### The exam will be graded for partial credit.

#### Sections 2-4 of the exam rely on the data midterm 1.xlsx, which gives total returns for 5 futures. You do not need to know anything about futures per se–just consider these to be valid return series on 5 assets.
#### This data file may be found in the class github repo, in the data folder. For your convenience, it is also posted on Canvas, in the Midterm exam.

# 1. Short Answer (30 pts)

#### No calculations or data analysis in this section; conceptual answers will suffice.

1. <b>(5 pts) Suppose an endowment is optimizing across multiple asset classes: equities, bonds, commodities, and currencies. They are considering whether to add cryptocurrencies as a new asset class for their mean-variance optimization. Do you think it is reasonable to make this decision based on whether the Sharpe ratio of crypto is higher than the Sharpe ratios of their current asset classes? Explain.</b><br><br>

<span style="color:#00008B"> **Solution:** While a higher sharpe ratio should definitely be a consideration for adding crypto to the endowment portfolio, a higher priority should be to make this decision based on the correlation of crypto with other asset classes. If crypto has lower correlations with other asset classes, even with lower sharpe ratio crypto could boost the overall sharpe ratio of the endowment.<br><br> The question might be interpreted as, conditional on adding crypto, should it be added as a new asset class or as part of the traditional currencies portfolio. This is a reasonable interpretation of the question, and the answer would be the same as the answer listed above for the more obvious interpretation. (Namely, the Sharpe ratio would be a poor guide to whether it should be included in currencies or put in a new asset class. Rather, the covariance matrix of cryptocurrencies to the other asset classes is much more relevant.)</span>
    
2. <b>(5 pts) True or False. (And explain your reason.)<br><br>
    We found that small changes in estimated mean returns have large impacts on the mean-variance frontier. (By “frontier”, we mean the set of achievable means and variances.)</b><br><br>

<span style="color:#00008B"> **Solution: <u>False</u>** The changes in estimated mean returns only have large impact <u> only on the weights of the mean-variance optimized portfolio and not on the frontier itself.</u> The MV solution (and not the frontier) is linear combination of the tangency portfolio and a riskless asset (GMV or risk-free rate). The reason for the sensitivity to the means is only partly about the means themselves and the tangency portfolio weights are sensitive to changes in mean return due to the nature of the inverted covariance matrix.</span>
    
3. <b> (5 pts) One might estimate the Value-at-Risk directly from the historic data, (the empirical cdf,)
or from using an approximation based on the normal distribution.<br><br>
    Which did we find performed better in actual data? How did we judge which performed better?</b><br><br>

<span style="color:#00008B"> **Solution:**  We found that the VaR ( at 5%) calculated using the approximation based on the normal distribution performed much better as it was able to provide the VaR estimate by frequency of days below the VaR value close to 5%. By definition of VaR, the number returns beyond the estimated VaR (5%) should be occuring with a frequency close to 5%. The notebook from Class Session 3 found that for many assets, the empirical CDF estimate of VaR led to a less accurate "hit ratio".</span>

4. <b> (5 pts) What approach does Harvard take to getting more realistic weights? What is a problem with this approach?</b><br><br>

<span style="color:#00008B"> **Solution:** Harvard places bounds on the portfolio allocation rather than implementing whatever numbers come out of the MV optimization problem along with a long only constraint on non-cash assets. The solution is numerical (rather than an explicit formula,) due to the inequality constraints. While the solution is computationally easy, it leads to the need for many boundary parameters, which greatly influence the solution. Thus, the solution may be overparameterized with little guide on how to set the parameters, or the motive to parameterize the problem to achieve a certain solution.</span>

5. <b> What aspect of tools such as Ridge regression and LASSO regression are useful in mean-variance optimization? That is, what problem in the classic MV solution are they addressing? </b><br><br>
    
<span style="color:#00008B"> **Solution:**  <span style="color:00008B">Both these tools limit positions from being too large, without specifying security-specific boundaries, and might prove better for out-of-sample performance by implementing a Bayesian approach to Mean-Variance optimization. Ridge and LASSO address the extereme weights produced by the mean-variance optimization by penalizing the objective function, (equivalently by regularizing the covariance matrix, which improves the stability of its inverse.) </span>

6. <b> 5pts) Consider three series: HFRIFWI, MLEIFCTR, and HDG. <br> Explain how they differ. That is, why were we interested in all three with regard to hedge-fund replication? </b><br><br>

<span style="color:#00008B"> **Solution:** 

<span style="color:#00008B"> MLEIFCTR attempts to replicate the HFRIFWI using a set of basic asset benchmarks, similar to what we did with our replicating regressions. HDG tracks a modified version of the ML Factor Model, but as a traded fund rather than a theoretical index. That is, it trades liquid securities suggested by the Factor Model, (slightly modified from MLEIFCTR.) Key to this question is demonstrating an understanding of survey, index, and traded fund.</span>
    
- <span style="color:#00008B"> **HFRIFWI** - Hedge fund Index Series. Index of surveyed hedge funds (2000 Holdings) designed to track the performance of the hedge market as a whole. Monthly returns based on surveys. <br><br></span>

- <span style="color:#00008B"> **MLEIFCTR** - Index Series created by Merrill Lynch, to replicate the performance of HFRI. Its a linear factor decomposition of the HFRI returns into 6 assets. This is not an asset and not an ETF. <br><br></span>

- <span style="color:#00008B"> **HDG** - Actual tradaeble asset trying to track the exchange series similar to MLEIFCTR, by actually trading on some liquid assets. HDG do not strictly adhere to the replication and would deviate from positions if they can get additional returns or reduce transaction cost. <br><br> </span>

# Imports

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis, skew
from scipy.stats import norm
import seaborn as sns
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (16,6)

# Helper Functions

In [2]:
def performance_summary(return_data, annualization = 12):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*annualization)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(annualization))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']

    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

In [3]:
def mvo_performance_stats(asset_returns,cov_matrix,port_weights, port_type,period):
    """ 
        Returns the Annualized Performance Stats for given asset returns, portfolio weights and covariance matrix
        Inputs: 
            asset_return - Excess return over the risk free rate for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
            port_weights = weights of the assets in the portfolio (1 x n) Vector
            port_type = Type of Portfolio | Eg - Tangency or Mean-Variance Portfolio
            period = Monthly frequency
    """
    
    ret = np.dot(port_weights,asset_returns)
    vol = np.sqrt(port_weights @ cov_matrix @ port_weights.T)*np.sqrt(period)
    sharpe = ret/vol

    stats = pd.DataFrame([[ret,vol,sharpe]],columns= ["Annualized Return","Annualized Volatility","Annualized Sharpe Ratio"], index = [port_type])
    return stats

In [4]:
def tangency_portfolio_rfr(asset_return,cov_matrix, cov_diagnolize = False):
    """ 
        Returns the tangency portfolio weights in a (1 x n) vector
        Inputs: 
            asset_return - return for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
    """
    if cov_diagnolize:
        asset_cov = np.diag(np.diag(cov_matrix))
    else:
        asset_cov = np.array(cov_matrix)
    inverted_cov= np.linalg.inv(asset_cov)
    one_vector = np.ones(len(cov_matrix.index))
    
    den = (one_vector @ inverted_cov) @ (asset_return)
    num =  inverted_cov @ asset_return
    return (1/den) * num

In [5]:
def mv_portfolio_rfr(asset_return,cov_matrix,target_ret,tangency_port):
    """ 
        Returns the Mean-Variance portfolio weights in a (1 x n) vector when a riskless assset is available
        Inputs: 
            asset_return - Excess return over the risk free rate for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
            target_ret = Target Return (Annualized)
            tangency_port = Tangency portfolio when a riskless assset is available
    """
    asset_cov = np.array(cov_matrix)
    inverted_cov= np.linalg.inv(asset_cov)
    one_vector = np.ones(len(cov_matrix.index))
    
    delta_den = (asset_return.T @ inverted_cov) @ (asset_return)
    delta_num = (one_vector @ inverted_cov) @ (asset_return)
    delta_tilde = (delta_num/delta_den) * target_ret
    return (delta_tilde * tangency_port)

In [6]:
def gmv_portfolio(asset_return,cov_matrix):
    """ 
        Returns the Global Minimum Variance portfolio weights in a (1 x n) vector
        Inputs: 
            asset_return - return for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
    """
    asset_cov = np.array(cov_matrix)
    inverted_cov= np.linalg.inv(asset_cov)
    one_vector = np.ones(len(cov_matrix.index))
    
    den = (one_vector @ inverted_cov) @ (one_vector)
    num =  inverted_cov @ one_vector
    return (1/den) * num

In [7]:
def mv_portfolio(asset_return,cov_matrix,target_ret,tangency_port):
    """ 
        Returns the Mean-Variance portfolio weights in a (1 x n) vector when no riskless assset is available
        Inputs: 
            asset_return - total return for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
            target_ret = Target Return (Not-Annualized)
            tangency_port = Tangency portfolio
    """
    omega_tan = tangency_portfolio_rfr(asset_return.mean(),cov_matrix)
    omega_gmv = gmv_portfolio(asset_return,cov_matrix) 
    
    mu_tan = asset_return.mean() @ omega_tan
    mu_gmv = asset_return.mean() @ omega_gmv
    
    delta = (target_ret - mu_gmv)/(mu_tan - mu_gmv)
    mv_weights = delta * omega_tan + (1-delta)*omega_gmv
    return mv_weights

In [8]:
def regression_based_performance(factor,fund_ret,rf,constant = True):
    """ 
        Returns the Regression based performance Stats for given set of returns and factors
        Inputs:
            factor - Dataframe containing monthly returns of the regressors
            fund_ret - Dataframe containing monthly excess returns of the regressand fund
            rf - Monthly risk free rate of return
        Output:
            summary_stats - (Beta of regression, treynor ratio, information ratio, alpha). 
    """
    if constant:
        X = sm.tools.add_constant(factor)
    else:
        X = factor
    y=fund_ret
    model = sm.OLS(y,X,missing='drop').fit()
    
    if constant:
        beta = model.params[1:]
        alpha = round(float(model.params['const']),6)
        
    else:
        beta = model.params
    treynor_ratio = ((fund_ret.values-rf.values).mean()*12)/beta[0]
    tracking_error = (model.resid.std()*np.sqrt(12))
    if constant:        
        information_ratio = model.params[0]*12/tracking_error
    r_squared = model.rsquared
    if constant:
        return (beta,treynor_ratio,information_ratio,alpha,r_squared,tracking_error)
    else:
        return (beta,treynor_ratio,r_squared,tracking_error)

In [9]:
def rolling_regression_param(factor,fund_ret,roll_window = 60):
    """ 
        Returns the Rolling Regression parameters for given set of returns and factors
        Inputs:
            factor - Dataframe containing monthly returns of the regressors
            fund_ret - Dataframe containing monthly excess returns of the regressand fund
            roll_window = rolling window for regression
        Output:
            params - Dataframe with time-t as the index and constant and Betas as columns
    """
    X = sm.add_constant(factor)
    y= fund_ret
    rols = RollingOLS(y, X, window=roll_window)
    rres = rols.fit()
    params = rres.params.copy()
    params.index = np.arange(1, params.shape[0] + 1)
    return params
    

In [10]:
def calc_probability_lowret(num_years,mean_ret_check,mean_ret,vol):
        """ 
        Returns the Probability that the cumulative market return will fall short of the cumulative
        risk-free return for each period
        Inputs: 
            mean - annualized mean returns of market for a period.
            vol - annualized volatility of returns for a period
            num_years - Number of years to calculate
        Output:
            probability - DataFrame with probability for each period (step = 1)
        """
        lst = []
        for n in range (0,num_years+1,1):
            norm_val = np.sqrt(n)*(mean_ret_check - mean_ret)/(vol)
            prob = (norm.cdf(norm_val))*100
            lst.append(pd.DataFrame([[n,prob]],columns=['Time','Probability(%)']))
        probability = pd.concat(lst)
        return probability

# Reading Data

In [11]:
total_ret = pd.read_excel(r'C:\Users\piyus\Documents\Python Scripts\data\midterm_1.xlsx',sheet_name = 'total returns', index_col = 'date')
total_ret.head()

,CL1,GC1,KC1,ES1,BP1
date,,,,,
2009-01-31,-0.113532,0.048627,0.061111,-0.086109,-0.007831
2009-02-28,0.044680,0.015187,-0.079479,-0.107294,-0.008309
2009-03-31,0.087892,-0.021111,0.034402,0.087209,0.001745
2009-04-30,-0.013826,-0.036543,-0.006491,0.094679,0.032897
2009-05-31,0.287437,0.098300,0.185518,0.055172,0.088934


## 2. Allocation (15 pts)

#### Consider mean-variance optimization using total returns. (That is, you are NOT analyzing excess returns. No need to subtract or otherwise consider a risk-free rate.)

### 2.1 (5pts) Report the weights of the Global Minimum Variance (GMV) portfolio, $ω^{v}$, and the weights of the Tangency portfolio, $ω^{t}$.

In [12]:
tr_performance = performance_summary(total_ret)
mean_total_ret =  np.array(tr_performance['Mean'])

tangency_port = tangency_portfolio_rfr(mean_total_ret,total_ret.cov())
gmv_port = gmv_portfolio(total_ret,total_ret.cov())
TangencyPort_df = pd.DataFrame(tangency_port,columns= ["Tangency Portfolio Weight"],index=tr_performance.index)
GMVPort_df = pd.DataFrame(gmv_port,columns= ["GMV Portfolio Weight"],index=total_ret.columns)
pd.concat([TangencyPort_df.T,GMVPort_df.T]).T

,Tangency Portfolio Weight,GMV Portfolio Weight
CL1,-0.128124,-0.030674
GC1,1.191087,0.179111
KC1,0.097813,-0.010639
ES1,4.220019,0.092154
BP1,-4.380796,0.770048


### 2.2 (5pts) What are the weights of the optimal portfolio, $ω^{∗}$, with a targeted total mean return of 0.20 annualized, (0.20/12 per month)?
### What is the volatility of this portfolio allocation?

In [27]:
target_ret = 0.20/12
mv_port= mv_portfolio(total_ret,total_ret.cov(),target_ret,tangency_port)

MVPort_df = pd.DataFrame(mv_port,columns= ["Mean-Variance Portfolio Weight"],index=tr_performance.index)
MVPort_df

,Mean-Variance Portfolio Weight
CL1,-0.059678
GC1,0.480307
KC1,0.021639
ES1,1.320735
BP1,-0.763003


In [14]:
mv_port_stats = mvo_performance_stats(mean_total_ret,total_ret.cov(),mv_port,'Mean-Variance Portfolio',12)
mv_port_stats

,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio
Mean-Variance Portfolio,0.2,0.190461,1.050082


<span style="color:#00008B">where **Sharpe** ratio in the table above and elsewhere is being used loosely given that we have total, not excess, returns.</span>

### 2.3 (5pts) Suppose we re-did this problem assuming the existence of a risk-free rate. Would the solution with mean return of 0.20 be guaranteed to have less than or equal volatility as the solution above, where we did not have a risk-free rate? Why?

<span style="color:#00008B"> **Solution:** The MVO portfolio with mean return of 0.20 be guaranteed will have **lower volatility with a risk-free return included** compared to the solution with no risk free rate. The MVO solution with no risk free rate combines GMV portfolio with tangency portfolio. Since, the risk free rate will always have lower volatility compared to GMV (which is formed by some combination of risky assets), the MVO solution with risk free rate will always have lower volatility. <br><br>
Conceptually, adding another asset to the menu, (the risk-free asset,) can only improve the available investment opportunities, (the MV frontier.)</span>

## 3. Hedging & Replication (20pts)

#### Consider replicating ES1 with BP1.
- #### include an intercept.
- #### estimate the hedge ratio over a rolling, 36-month window.
- #### For each month, you should have the optimal replicating position, which allows you to properly invest in BP1 (and a constant) to replicate ES1.
- #### Then calculate the return of that investment, which is the fitted regression value

#### Thus, you should have a time-series of the replication results. Call this series, $r^{∗}$.

In [15]:
factor =  pd.DataFrame(total_ret['BP1'].copy())
fund_ret = pd.DataFrame(total_ret['ES1'].copy())
rolling_reg = rolling_regression_param(factor,fund_ret,roll_window = 36)
rolling_reg.index = factor.index
factor_ret = factor.loc[rolling_reg.dropna().index]
one_df = pd.DataFrame(np.ones(factor_ret.shape[0])).T
one_df.columns = factor_ret.index
factor_ret = pd.concat([one_df,pd.DataFrame(factor_ret).T])
rolling_reg_replication = pd.DataFrame(np.diag(pd.DataFrame(np.array(rolling_reg[rolling_reg.dropna().index.min():]) @ np.array(factor_ret))),index = factor_ret.columns,columns = ['ES1'])

### 3.1 (5pts) Report the last five values of the timeseries of alpha and beta estimates.

In [17]:
rolling_reg_params = rolling_reg.copy()
rolling_reg_params.index = total_ret.index
rolling_reg_params.tail(5)

,const,BP1
date,,
2022-02-28,0.013660,1.178892
2022-03-31,0.014433,1.144168
2022-04-30,0.012270,1.247557
2022-05-31,0.013008,1.186588
2022-06-30,0.010191,1.251756


### 3.2 (5pts) Report the last five values of the timeseries, $r^{∗}$.

In [67]:
rolling_reg_replication.rename(columns = {'ES1':'In-Sample $r^{*}$'}).tail(5)

,In-Sample $r^{*}$
date,
2022-02-28,0.010155
2022-03-31,-0.009789
2022-04-30,-0.038274
2022-05-31,0.013573
2022-06-30,-0.030127


#### In case the question was interpreted as getting Out of sample returns

In [68]:
oos_returns = pd.DataFrame(np.diag(pd.DataFrame(np.array(rolling_reg[rolling_reg.dropna().index.min():].shift()) @ np.array(factor_ret))),index = factor_ret.columns,columns = ['ES1'])
oos_returns.dropna().rename(columns = {'ES1':'Out-Of-Sample $r^{*}$'}).tail(5)

,Out-Of-Sample $r^{*}$
date,
2022-02-28,0.011405
2022-03-31,-0.011298
2022-04-30,-0.031923
2022-05-31,0.012864
2022-06-30,-0.025211


### 3.3 (5pts) Report the mean, volatility and mean-to-vol ratio of this replicated position, $r^{∗}$. Annualize the statistics.

In [69]:
rolling_reg_performance = performance_summary(rolling_reg_replication.rename(columns = {'ES1':'In-Sample $r^{*}$'}))
rolling_reg_performance

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Max Drawdown,Peak,Bottom,Recovery
In-Sample $r^{*}$,0.126333,0.062546,2.019829,-0.35217,0.515144,-0.023069,-0.031456,-0.063843,2022-02-28,2022-06-30,NaT


In [70]:
oos_rolling_reg_performance = performance_summary(oos_returns.rename(columns = {'ES1':'Out-Of-Sample $r^{*}$'}))
oos_rolling_reg_performance

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Max Drawdown,Peak,Bottom,Recovery
Out-Of-Sample $r^{*}$,0.128111,0.062769,2.040989,-0.405036,1.163322,-0.022947,-0.030451,-0.054995,2016-05-31,2016-10-31,2017-03-31


### 3.4 (5pts) How correlated is this replicated position, $r^{∗}$, to ES1?

In [71]:
print('Correlation of In-Sample Rolling Regression Fitted Returns with Actual Returns : ' + str(np.round(np.corrcoef(total_ret.loc[rolling_reg.dropna().index]['ES1'], rolling_reg_replication['ES1'])[0,1],5))+'\n\n')
print('Correlation of OOS Rolling Regression Fitted Returns with Actual Returns : ' + str(np.round(np.corrcoef(total_ret.loc[oos_returns.dropna().index]['ES1'], oos_returns['ES1'].dropna())[0,1],5)))


Correlation of In-Sample Rolling Regression Fitted Returns with Actual Returns : 0.52104


Correlation of OOS Rolling Regression Fitted Returns with Actual Returns : 0.4159


## 4. Modeling Risk (35pts)

####  Consider the return series of a portfolio invested 50% in ES1 (stocks) and 50% in GC1 (gold).
- #### Build a return stream with weights of 0.5 in EC1 and 0.5 in GC1.
- #### We will consider total returns—no need to subtract a risk-free rate.

In [21]:
ES1_GC1_returns = (0.5 * total_ret['ES1'] + 0.5 * total_ret['GC1'] ).to_frame('ES1-GC1 50-50 Returns')

### 4.1 (10pts) Compare the performance statistics between ES1, GC1, and the 50/50 portfolio. Annualize mean and volatility.
- ### Mean
- ### Volatility
- ### Mean divided by volatility.
- ### Maximum Drawdown

In [22]:
combined = pd.concat([total_ret.loc[:,['ES1']].T,total_ret.loc[:,['GC1']].T,ES1_GC1_returns.T]).T
perf_summary = performance_summary(combined)
perf_summary

,Mean,Volatility,Sharpe Ratio,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05),Max Drawdown,Peak,Bottom,Recovery
ES1,0.129117,0.151067,0.854698,-0.439491,0.723828,-0.071244,-0.091453,-0.203174,2019-12-31,2020-03-31,2020-07-31
GC1,0.054008,0.160134,0.337267,0.113357,0.142579,-0.065664,-0.085920,-0.429597,2011-08-31,2015-12-31,NaT
ES1-GC1 50-50 Returns,0.091562,0.114079,0.802619,0.109368,0.253804,-0.045782,-0.058951,-0.120982,2022-03-31,2022-06-30,NaT


### 4.2 (5pts) How does the 50/50 portfolio compare to the individual components in terms of
- ### volatility
- ### maximum drawdown
- ### mean-to-vol ratio
### Is this as we would expect from the portfolio theory?

<span style="color:#00008B"> **Solution:** The 50-50 combined portfolio has lower volatility and Max Drawdown and higher Mean-to-Vol ratio (Sharpe Ratio). Portfolio Theory dictates that combining assets with less than perfect correlation the portfolio risk goes down. This **diversification** is what the theory suggests: volatility is less than the sum of its parts, and mean-vol ratio increases. The Maximum Drawdown shows a similar diversification, which is not guaranteed by theory but is typical empirically. Thus, this result is expected.</span>

### 4.3 (10pts) Calculate the 1-period, 1% CVaR of the 50/50 portfolio analytically.

- ### Use the normality formula.
- ### Ignore the mean in the formula—set it to 0.

### Calculate the CVaR in two different ways:

### 4.3.a) Use the full-sample volatility estimate.

In [23]:
CVaR_full_sample = -(norm.pdf(2.33)/0.01  * ES1_GC1_returns.std())
CVaR_full_sample.to_frame('Full Sample CVar')           

,Full Sample CVar
ES1-GC1 50-50 Returns,-0.087027


### 4.3.b)  Use the latest value from the rolling 24-month volatility.

In [24]:
sigma_rolling = ES1_GC1_returns.shift(1).rolling(24).std().dropna()
rolling_cvar = -(norm.pdf(2.33)/0.01 * sigma_rolling)
rolling_cvar.rename(columns={'ES1-GC1 50-50 Returns':'Rolling CVaR'},inplace =True)
rolling_cvar.tail(1)

,Rolling CVaR
date,
2022-06-30,-0.090833


### 4.4 (10pts) What is the probability that over the next 10 years, ES1 underperforms the 50/50 portfolio?

### To answer the question,
- ### use statistical estimates of the returns over the full sample.
- ### Assume that log returns for both assets are normally distributed.

In [80]:
ES1_mean = np.log(1 + combined.ES1).mean() * 12
Port1_mean = np.log(1 + combined['ES1-GC1 50-50 Returns']).mean() * 12
sigma = (np.log(1 + combined.ES1) - np.log(1 + combined['ES1-GC1 50-50 Returns'])).std() * np.sqrt(12)
mu = ES1_mean - Port1_mean

probability = calc_probability_lowret(10,0,mu,sigma)
probability = probability.set_index('Time')
probability

,Probability(%)
Time,
0,50.000000
1,38.064680
2,33.373847
3,29.938619
4,27.173801
5,24.848077
6,22.840433
7,21.077561
8,19.510774
